# Natural Language Processing with ConvNets

 The goal of this challenge is to show you that we can also use 1D Convolutional Filters to scan sentences  (_instead of RNN_).

 Run this cell 

In [ ]:
!pip freeze | grep gensim

gensim==4.1.2


# The data

 **Question**  Let's first load the data. You don't have to understand what is going on in the function, it does not matter here.

 **Warning**  The `load_data` function has a `percentage_of_sentences` argument. Depending on your computer, there are chances that too many sentences will make your compute slow down, or even freeze - your RAM can overflow. For that reason, **you should start with 10% of the sentences** and see if your computer handles it. Otherwise, rerun with a lower number. 


In [ ]:

### Run this cell to load the data ###


import tensorflow_datasets as tfds
from tensorflow.keras.preprocessing.text import text_to_word_sequence

def load_data(percentage_of_sentences=None):
    train_data, test_data = tfds.load(name="imdb_reviews", split=["train", "test"], batch_size=-1, as_supervised=True)

    train_sentences, y_train = tfds.as_numpy(train_data)
    test_sentences, y_test = tfds.as_numpy(test_data)
    
    # Take only a given percentage of the entire data
    if percentage_of_sentences is not None:
        assert(percentage_of_sentences> 0 and percentage_of_sentences<=100)
        
        len_train = int(percentage_of_sentences/100*len(train_sentences))
        train_sentences, y_train = train_sentences[:len_train], y_train[:len_train]
  
        len_test = int(percentage_of_sentences/100*len(test_sentences))
        test_sentences, y_test = test_sentences[:len_test], y_test[:len_test]
    
    X_train = [text_to_word_sequence(_.decode("utf-8")) for _ in train_sentences]
    X_test = [text_to_word_sequence(_.decode("utf-8")) for _ in test_sentences]
    
    return X_train, y_train, X_test, y_test

X_train, y_train, X_test, y_test = load_data(percentage_of_sentences=10)

2022-12-15 16:17:11.904356: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-15 16:17:11.933812: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)


Remember that to do NLP, you need to go through one of the following option.
But in both cases, you can replace the recurrent layer (top part) by a CNN layer. 

We will try both options, starting from the one on the left were the embeddings are learned within the network.

# Part 1 : Concatenate a Keras Embedding with a Conv1D 

Let's train a fancy network here. 

Each of your words is represented by a vector of size N (the size of your embedding). Therefore, as a sentence is a sequence of words, it is represented by a matrix (number of words, N). Consequently, all your sentences are actually represented as matrices once embedded.

If you think about it, an image is also a matrix. Said differently, you may represent your sentence of word as a matrix, where each column (or row, depending on how you want to look at it) is a word, and each row (or each column) corresponds to a coordinate in the embedding space. 

Well, in that case, as these are close to images, why not using convolutions on them? Yes, convolutions!
But, be careful. In the case of images, convolutions are 2-dimensional as the filters scan these pictures left to right and top to bottom. In the case of our sentences, we want the kernel to move _only_ in word by word, hence left to right. It wouldn't make sense to move the kernels top to bottom as _one vector = one word_.

So let's create a model that uses convolutions.

## First, the data

 **Question**  You will need to prepare your data. First, tokenize them. Then, you need to pad them (use a value `maxlen` equal to 150). You also might need to compute the size of your vocabulary ;)

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)
    
# We apply the tokenization to the train and test set
X_train_token = tokenizer.texts_to_sequences(X_train)
X_test_token = tokenizer.texts_to_sequences(X_test)


X_train_pad = pad_sequences(X_train_token, maxlen=150, dtype='float32')
X_test_pad = pad_sequences(X_test_token, maxlen=150, dtype='float32')

vocab_size = len(tokenizer.word_index)
vocab_size

## Using 1D Convolution.

 **Question**  Define a model that has :
- an `Embedding` layer: 
    - `input_dim` = `vocab_size + 1`
    - `output_dim` is the embedding space dimension
    - `mask_zero` has to be set to `True`. 
    - Here, for computational reasons, set `input_length` to the maximum length of your observations (that you just defined in the previous question).
- a `Conv1D` layer 
- a `Flatten` layer
- a `Dense` layer
- an output layer

Compile the model accordingly.



In [ ]:
from tensorflow.keras import Sequential, layers

def init_cnn_model(vocab_size):
    model = Sequential()
    model.add(layers.Embedding(input_dim=vocab_size + 1, output_dim=10, mask_zero=True, input_length=150))
    model.add(layers.Conv1D(16, 3))
    model.add(layers.Flatten())
    model.add(layers.Dense(5,))
    model.add(layers.Dense(1, activation='sigmoid'))

    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

model_cnn = init_cnn_model(vocab_size)

 **Question**  Look at the number of parameters. You can compare it to the model that you had in previous exercises (esp. the first one)

In [ ]:
model_cnn.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 150, 10)           304200    
_________________________________________________________________
conv1d (Conv1D)              (None, 148, 16)           496       
_________________________________________________________________
flatten (Flatten)            (None, 2368)              0         
_________________________________________________________________
dense (Dense)                (None, 5)                 11845     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 6         
Total params: 316,547
Trainable params: 316,547
Non-trainable params: 0
_________________________________________________________________


 **Question**  Fit your model with a stopping criterion, and evaluate it on the test data.

You will probably notice that it is ... **much faster** than RNN.

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

es = EarlyStopping(patience=5, restore_best_weights=True)

model_cnn.fit(X_train_pad, y_train, 
          epochs=20, 
          batch_size=32,
          validation_split=0.3,
          callbacks=[es]
         )


res = model_cnn.evaluate(X_test_pad, y_test, verbose=0)

print(f'The accuracy evaluated on the test set is of {res[1]*100:.3f}%')

Epoch 1/20
55/55 [==============================] - 1s 4ms/step - loss: 0.6928 - accuracy: 0.5120 - val_loss: 0.6930 - val_accuracy: 0.5280
Epoch 2/20
55/55 [==============================] - 0s 2ms/step - loss: 0.5989 - accuracy: 0.8509 - val_loss: 0.6727 - val_accuracy: 0.5787
Epoch 3/20
55/55 [==============================] - 0s 2ms/step - loss: 0.2839 - accuracy: 0.9634 - val_loss: 0.5973 - val_accuracy: 0.6880
Epoch 4/20
55/55 [==============================] - 0s 2ms/step - loss: 0.0544 - accuracy: 0.9954 - val_loss: 0.5575 - val_accuracy: 0.7213
Epoch 5/20
55/55 [==============================] - 0s 2ms/step - loss: 0.0144 - accuracy: 1.0000 - val_loss: 0.5710 - val_accuracy: 0.7227
Epoch 6/20
55/55 [==============================] - 0s 2ms/step - loss: 0.0077 - accuracy: 0.9989 - val_loss: 0.5715 - val_accuracy: 0.7227
Epoch 7/20
55/55 [==============================] - 0s 2ms/step - loss: 0.0039 - accuracy: 1.0000 - val_loss: 0.5794 - val_accuracy: 0.7267
Epoch 8/20
55/55 [==

# Part 2 : Learn a Word2Vec representation, and then feed it into a NN with a `Conv1D` 



In the first part of the exercise, you were asked to jointly learn the embedding representation and the CNN convolution within the same network.

Now, let's try to replace the RNN with a CNN for an architecture, as shown on the right side.



 **Question**  Learn a Word2Vec model or load a trained one directly from GENSIM (transfer learning). Then, prepare your data as in the previous exercise. This question is quite long but it prepares you for real=world challenges. Don't worry, you have already done all the building bricks in the previous exercises!

In [ ]:
import gensim.downloader as api
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

# Load a Word2Vec embedding
word2vec_transfer = api.load("glove-wiki-gigaword-50")

# Function to convert a sentence (list of words) into a matrix representing the words in the embedding space
def embed_sentence_with_TF(word2vec, sentence):
    embedded_sentence = []
    for word in sentence:
        if word in word2vec:
            embedded_sentence.append(word2vec[word])
        
    return np.array(embedded_sentence)

# Function that converts a list of sentences into a list of matrices
def embedding(word2vec, sentences):
    embed = []
    
    for sentence in sentences:
        embedded_sentence = embed_sentence_with_TF(word2vec, sentence)
        embed.append(embedded_sentence)
        
    return embed

# Embed the training and test sentences
X_train_embed_2 = embedding(word2vec_transfer, X_train)
X_test_embed_2 = embedding(word2vec_transfer, X_test)

# Pad the training and test embedded sentences
X_train_pad_2 = pad_sequences(X_train_embed_2, dtype='float32', padding='post', maxlen=200)
X_test_pad_2 = pad_sequences(X_test_embed_2, dtype='float32', padding='post', maxlen=200)

[==================================================] 100.0% 66.0/66.0MB downloaded


 **Question**  Now, build a model that has a `Conv1D` layer, a `Flatten` layer, a `Dense` layer, and an `output` layer. Compile and fit it on the train data. You can then evaluate it on the test set.

In [ ]:
def init_cnn_model_2():
    model = Sequential()
    model.add(layers.Conv1D(16, 3))
    model.add(layers.Flatten())
    model.add(layers.Dense(5,))
    model.add(layers.Dense(1, activation='sigmoid'))

    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

model_cnn_2 = init_cnn_model_2()


es_2 = EarlyStopping(patience=5, restore_best_weights=True)

model_cnn_2.fit(X_train_pad_2, y_train, 
          epochs=20, 
          batch_size=32,
          validation_split=0.3,
          callbacks=[es_2]
         )


res = model_cnn_2.evaluate(X_test_pad_2, y_test, verbose=0)

print(f'The accuracy evaluated on the test set is of {res[1]*100:.3f}%')

Epoch 1/20
55/55 [==============================] - 0s 3ms/step - loss: 0.7760 - accuracy: 0.5451 - val_loss: 0.7068 - val_accuracy: 0.5907
Epoch 2/20
55/55 [==============================] - 0s 2ms/step - loss: 0.4881 - accuracy: 0.7691 - val_loss: 0.6827 - val_accuracy: 0.6267
Epoch 3/20
55/55 [==============================] - 0s 2ms/step - loss: 0.3645 - accuracy: 0.8571 - val_loss: 0.7069 - val_accuracy: 0.6587
Epoch 4/20
55/55 [==============================] - 0s 2ms/step - loss: 0.2700 - accuracy: 0.9166 - val_loss: 0.7811 - val_accuracy: 0.6320
Epoch 5/20
55/55 [==============================] - 0s 2ms/step - loss: 0.2070 - accuracy: 0.9451 - val_loss: 0.8339 - val_accuracy: 0.6440
Epoch 6/20
55/55 [==============================] - 0s 2ms/step - loss: 0.1438 - accuracy: 0.9743 - val_loss: 0.9477 - val_accuracy: 0.6360
Epoch 7/20
55/55 [==============================] - 0s 2ms/step - loss: 0.1039 - accuracy: 0.9846 - val_loss: 1.0044 - val_accuracy: 0.6400
The accuracy evaluat

 **Question**  You might be frustrated by the accuracy you got, this happens to us all at some point. Once you have tested your first iteration, you need to improve your models: by making them more complex, changing the parameters, stacking additional layers, and so on...

Only practice and experimentation will get you there. So you can go back to your previous models, change them and try to get better results ;)